This code is not commercial and created only for study purpose. The main aim is introducing users of HPC-DA system of TU-Dresden with TensorFlow and Jupyter notebook. This code does not pretend on completeness of research significance. Fell free to use and improve this code for your study purposes.

Importing PyTorch and all required libraries and checking the working capacity

In [ ]:
import torch
import torchvision
import numpy as np
from torch import nn
from torch import optim
from torchvision import models
from torchvision import transforms
from torchvision import datasets
from time import time

torch.version.__version__

Clean the data. Convert images into numbers (tensor) and normalize the tensor. Download the MNIST dataset directly from API

Load dataset to Data loader. It combines a dataset and a sampler and provides an iterable over the given dataset

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
MNIST_data_train = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, target_transform=None, download=True)
MNIST_data_validate = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(MNIST_data_train,
                                          batch_size=32,
                                          shuffle=True,
                                          num_workers=2)
validate_loader = torch.utils.data.DataLoader(MNIST_data_train,
                                          batch_size=32,
                                          shuffle=True,
                                          num_workers=2)

Build The Neural Network. Copy the neural network from the Neural Networks section

In [ ]:
model = nn.Sequential(nn.Linear(28*28, 512),
                      nn.ReLU(),
                      nn.Linear(512, 512),
                      nn.ReLU(),
                      nn.Linear(512, 10),
                      nn.LogSoftmax(dim=1))
print(model)

Cross-entropy loss

In [ ]:
criterion = nn.NLLLoss()
images, labels = next(iter(train_loader))
images = images.view(images.shape[0], -1)

logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

Get some random training images. Show the shape. 

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

Backpropogation. Adjust the weights of the network to minimize the loss

In [ ]:
print('Originally: \n', model[0].weight.grad)
loss.backward()
print('After backpropogation: \n', model[0].weight.grad)

Optimize the model, do gradient descent and update the weights by back-propagation

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0003)
time0 = time()
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images = images.view(images.shape[0], -1)
    
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_loader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Evaluate the model and calculate the accuracy

In [ ]:
correct_count, all_count = 0, 0
for images,labels in validate_loader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Save the model. It will save the already trained model for future use

In [ ]:
def saving_model():
    torch.save(model, './example3_MNIST.pt')
    return "Model saved"
saving_model()